In [6]:
!pip install ImbalancedLearningRegression


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import pandas as pd
# from imbalanced_regression import smogn
import ImbalancedLearningRegression
import smogn

# Load your data
df = pd.read_csv("processed.csv")

# Apply SMOGN to handle the imbalance in 'Goals'
df_smogn = smogn.smoter(
    data=df,
    y='Goals',               # Target column
    k=5,                     # Number of neighbors for generating synthetic samples
    samp_method='balance',   # Balances over-sampling and under-sampling
    under_samp=0.6,          # Under-sampling proportion
    rel_thres=0.8            # Threshold to detect outliers
)

# Separate features and target after SMOGN
y_smogn = df_smogn["Goals"]
X_smogn = df_smogn.drop(columns=["Goals"])

# Proceed with scaling, train-test split, and modeling as usual
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Scale numeric features
numeric_columns = X_smogn.select_dtypes(include=['number']).columns
scaler = StandardScaler()
X_smogn[numeric_columns] = scaler.fit_transform(X_smogn[numeric_columns])

# Split the augmented dataset
X_train_smogn, X_test_smogn, y_train_smogn, y_test_smogn = train_test_split(X_smogn, y_smogn, test_size=0.2, random_state=42)

# Train Random Forest on the augmented data
from sklearn.ensemble import RandomForestRegressor

rf_smogn = RandomForestRegressor(random_state=42)
rf_smogn.fit(X_train_smogn, y_train_smogn)

# Evaluate on the test set
y_pred_smogn = rf_smogn.predict(X_test_smogn)

# Calculate metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error

mse_smogn = mean_squared_error(y_test_smogn, y_pred_smogn)
mae_smogn = mean_absolute_error(y_test_smogn, y_pred_smogn)
print(f"Mean Squared Error after SMOGN: {mse_smogn}")
print(f"Mean Absolute Error after SMOGN: {mae_smogn}")

/Users/ericmckevitt/Desktop/Stuff/Code/Mines/CSCI575/Soccer Goals Prediction/ml_proj1/lib/python3.12/site-packages/smogn/phi.py:81: RuntimeWarning: divide by zero encountered in scalar divide
  delta.append((y_rel[i + 1] - y_rel[i]) / h[i])


ValueError: redefine phi relevance function: all points are 0